In [7]:
#Imports
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *
import pyspark.sql.functions as f

In [8]:
#Criar sessão no spark
# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/projeto/nba'

builder = SparkSession \
    .builder \
    .master("local[2]") \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [ ]:
#Apagar tabela Silver da Base de Dados
spark.sql(
"""
drop table if exists nba.sl_players
"""
)

In [ ]:
#Apagar ficheiros da tabela no HDFS
from os import PathLike
from hdfs import InsecureClient

client = InsecureClient("http://hdfs-nn:9870", user="anonymous")

#delete_path = "hdfs://hdfs-nn:9000/projeto/nba/sl_players"
client.delete(delete_path_exp, True)

In [9]:
#Carregar tabela para dataframe
table_location = "hdfs://hdfs-nn:9000/projeto/nba/bz_players"
df = spark.read.format("delta").load(table_location)
#df.toPandas()

In [10]:
#Apagar colunas que nao fazem sentido com o use case
df1 = df.drop("_c0","GS","FTR","ORB%","DRB%","TRB%","AST%","STL%",
              "BLK%","TOV%","blanl","USG%","DWS","WS/48","blank2","DBPM",
              "BPM","VORP","FGA","FG%","3P%","2P%","eFG%","FT","FTA",
              "FT%","ORB","DRB","TRB","AST","STL","BLK",
              "TOV","PF")
#apagamos 33, ficamos com 20

In [11]:
#Eliminar os * na coluna do nome dos jogadores
df2 = df1.withColumn(
    "Player",
    f.regexp_replace(f.col("Player"), "[*]", "")
)

#df2.show()

In [12]:
#Criar coluna com total de jogos
df3= df2.withColumn(    
    "Pontos3P",
    col("3P")*3
)

df4 = df3.withColumn(
    "Pontos2P",
    col("2P")*2
)

df5 = df4.withColumn (
    "%Pontos3P",
    col("3P")/col("PTS")*100
)

#df5.show()

In [13]:
#Uniformizar siglas que nao estao corrretas apos join
from pyspark.sql.functions import when
dfnew = df5.withColumn(
        "Tm",
    when(col("Tm") == "GSW", "GST")\
    .when( col("Tm") == "SAS", "SAN")\
    .when( col("Tm") == "NOJ", "UTA")\
    .when( col("Tm") == "PHO", "PHX")\
    .otherwise(col("Tm"))
)

#dfnew.select("Tm").show(24691)

In [14]:
#Carregar tabela das abreviacoes para dar join
#Carregar tabela para dataframe
location1 = "hdfs://hdfs-nn:9000/projeto/nba/bz_abrev"
df_abrev = spark.read.format("delta").load(location1)
#df_abrev.toPandas()

In [15]:
#Join
cond = [df_abrev.sigla == dfnew.Tm]
df6= df_abrev.join(dfnew,cond, 'fullouter')
#df6.select("sigla","equipa","liga","Player","Tm").show(24848)

In [16]:
#Apos o join
dfnew1 = df6.withColumn(
        "equipa",
    when(col("Tm") == "TOT", "Varias Equipas")\
    .when( col("Tm") == "BRK", "Brooklyn Nets")\
    .when( col("Tm") == "OKC", "Oklahoma City Thunder")\
    .when( col("Tm") == "WSB", "Washington Wizards")\
    .when( col("Tm") == "STB", "St.Louis Bombers")\
    .when( col("Tm") == "DNN", "Denver Nuggets")\
    .otherwise(col("equipa"))
)

dfnew1 = dfnew1.withColumn(
        "sigla",
    when(col("Tm") == "TOT", "TOT")\
    .when(col("Tm") == "WSB", "WSB")
    .otherwise(col("sigla"))
)

#Dar o valor NBA à coluna Liga
dfnew1 = dfnew1.withColumn(
        "liga",
    when(col("Tm") == "GST", "NBA")\
    .when(col("Tm") == "SAN", "NBA")\
    .when(col("Tm") == "UTA", "NBA")\
    .when(col("Tm") == "PHX", "NBA")\
    .when(col("Tm") == "TOT", "NBA")\
    .when(col("Tm") == "BRK", "NBA")\
    .when(col("Tm") == "OKC", "NBA")\
    .when(col("Tm") == "WSB", "NBA")\
    .when(col("Tm") == "STB", "NBA")\
    .when(col("Tm") == "DNN", "NBA")\
    .otherwise(col("liga"))
)

dfnew1 = dfnew1.withColumn(
    "sigla",
    when(col("equipa") == "Brooklyn Nets", "BKN")
    .otherwise(col("sigla"))
)

dfnew1 = dfnew1.withColumn(
    "Tm",
    when(col("equipa") == "Brooklyn Nets", "BKN")\
    .otherwise(col("Tm"))
)

#Charlotte Hornets CHO
dfnew1 = dfnew1.withColumn(
    "equipa",
    when(col("Tm") == "CHO", "Charlotte Hornerts")\
    .otherwise(col("equipa"))
)

dfnew1 = dfnew1.withColumn(
    "sigla",
    when(col("Tm") == "OKC", "OKC")\
    .otherwise(col("sigla"))
)

#dfnew1.select("sigla","equipa","liga","Player","Tm").show(24848)

In [17]:
#Eliminar colunas sigla
df7 = dfnew1.drop("Tm")\
    .withColumnRenamed("liga","Liga")\
    .withColumnRenamed("equipa","Equipa")\
    .withColumnRenamed("sliga","Sigla")\
    .withColumnRenamed("G","Jogos")\
    .withColumnRenamed("Age","Idade")\
    .withColumnRenamed("MP","Minutos")\
    .withColumnRenamed("FG","TotalCestos")\
    .withColumnRenamed("PTS","Pontos")\
    .withColumnRenamed("Year","Ano")\
    .withColumnRenamed("Player","Jogador")\
    .withColumnRenamed("POS","Posicao")

#df7.printSchema()

In [18]:
df8 = df7.select("Ano","Equipa","Sigla","Liga","Jogador","Idade","Posicao","Jogos","Minutos","Pontos","TotalCestos","Pontos3P","3P","3PA","%Pontos3P","Pontos2P","2P","2PA","PER","TS%","3PAR","OWS","WS","OBPM")
#df8.printSchema()

In [19]:
from pyspark.sql.types import StringType
df8 = df8.withColumn("Ano", df8["Ano"].cast(StringType()))
#df8.toPandas()

In [20]:
#Uniformizar coluna das posicoes
from pyspark.sql.functions import when, col, concat, lit

df9 = df8.withColumn(
    "Posicao",
    when( (col("Posicao") == "PF") | (col("Posicao") == "PF-SF"), "Power Forward")\
    .when( (col("Posicao") == "SF") | (col("Posicao") == "SF-PF") | (col("Posicao") == "SF-SG"), "Small Forward" )\
    .when( (col("Posicao") == "SG") | (col("Posicao") == "SG-PG") | (col("Posicao") == "SG-SF"), "Shooting Guard" )\
    .when( (col("Posicao") == "PG") | (col("Posicao") == "PG-SG"), "Point Guard" )\
    .when( (col("Posicao") == "C") | (col("Posicao") == "C-PF"), "Center" )\
    .otherwise("Outra")
)

#df9.toPandas()

In [ ]:
#Create Table in DataBase
spark.sql(
"""
drop table if exists nba.sl_players
"""
)

spark.sql(
    """
    CREATE EXTERNAL TABLE nba.sl_players (
        Equipa STRING, Sigla STRING, Liga STRING, Jogador STRING, Idade INT, Posicao STRING, Jogos INT,
        Minutos INT, Pontos INT, TotalCestos INT, Pontos3P INT, 3P INT, 3PA INT, Perc_Pontos3P DOUBLE,
        Pontos2P INT, 2P INT, 2PA INT, PER DOUBLE,
        TS_Perc DOUBLE, 3PAR DOUBLE, OWS DOUBLE, WS DOUBLE, OBPM DOUBLE
    )
    USING DELTA
    PARTITIONED BY(
    Ano INT
    )
    TBLPROPERTIES (
    'skip.header.line.count'='1'
    )
    LOCATION 'hdfs://hdfs-nn:9000/projeto/nba/sl_players'
    """
)

In [ ]:
#Guardar o ultimo DataFrama (df9) na hivetable de silver

df9.select("*")\
    .write\
    .mode("overwrite")\
    .option("overwriteSchema","true")\
    .partitionBy("Ano") \
    .format("delta")\
    .save("hdfs://hdfs-nn:9000/projeto/nba/sl_players")

In [21]:
spark.sql(
"""
select * from nba.sl_players
"""
).show()

+----+--------------+-----+----+----------------+-----+--------------+-----+-------+------+-----------+--------+---+---+------------------+--------+---+----+----+-----+-----+----+----+----+
| Ano|        Equipa|Sigla|Liga|         Jogador|Idade|       Posicao|Jogos|Minutos|Pontos|TotalCestos|Pontos3P| 3P|3PA|         %Pontos3P|Pontos2P| 2P| 2PA| PER|  TS%| 3PAR| OWS|  WS|OBPM|
+----+--------------+-----+----+----------------+-----+--------------+-----+-------+------+-----------+--------+---+---+------------------+--------+---+----+----+-----+-----+----+----+----+
|2011|Varias Equipas|  TOT| NBA|   Alexis Ajinca|   22|        Center|   34|    340|   143|         59|      36| 12| 34| 8.391608391608392|      94| 47|  99|12.2|0.507|0.256| 0.0| 0.4|-3.1|
|2011|Varias Equipas|  TOT| NBA|  David Andersen|   30| Power Forward|   40|    373|   134|         56|      24|  8| 23| 5.970149253731343|      96| 48|  98|10.9|0.513| 0.19|-0.1| 0.3|-4.0|
|2011|Varias Equipas|  TOT| NBA| Carmelo Anthony| 